In [1]:
import requests
import time
import datetime
import pandas as pd

In [2]:
resp = requests.get('https://min-api.cryptocompare.com/data/pricemulti?fsyms=BTC&tsyms=USD')

In [3]:
resp.json()

{'BTC': {'USD': 4039.4}}

In [4]:
resp.json()['BTC']['USD']

4039.4

In [5]:
time.sleep(10)
print('hey')

hey


In [6]:
datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S')

'20-03-2019 22:30:03'

In [7]:
df = pd.DataFrame(columns=['Date', 'Value'])

In [8]:
df.columns

Index(['Date', 'Value'], dtype='object')

In [9]:
df.head()

,Date,Value


In [10]:
date = datetime.datetime.now().strftime('%d-%m-%Y %H:%M')

In [11]:
price = resp.json()['BTC']['USD']

In [12]:
df = df.append({'Date': date, 'Value': price}, ignore_index=True)

In [13]:
df.head()

,Date,Value
0,20-03-2019 22:30,4039.4


In [14]:
def data_retriever(sleep=30):
    global df
    while True:
        resp = requests.get('https://min-api.cryptocompare.com/data/pricemulti?fsyms=BTC&tsyms=USD')
        price = resp.json()['BTC']['USD']
        date = datetime.datetime.now().strftime('%d-%m-%Y %H:%M')
        df = df.append({'Date': date, 'Value': price}, ignore_index=True)
        print(df.shape)
        df.to_csv('data/btc-daily.csv', index=False)
        time.sleep(sleep)

In [15]:
data_retriever()

(2, 2)
(3, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)
(8, 2)
(9, 2)
(10, 2)
(11, 2)
(12, 2)
(13, 2)
(14, 2)
(15, 2)
(16, 2)


ConnectionError: HTTPSConnectionPool(host='min-api.cryptocompare.com', port=443): Max retries exceeded with url: /data/pricemulti?fsyms=BTC&tsyms=USD (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f7cb54ac470>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [16]:
df.shape

(10, 2)

In [17]:
df.head()

,Date,Value
0,20-03-2019 21:53,4032.37
1,20-03-2019 21:54,4036.70
2,20-03-2019 21:54,4036.21
3,20-03-2019 21:55,4035.83
4,20-03-2019 21:56,4036.39


In [18]:
df.tail()

,Date,Value
5,20-03-2019 21:56,4035.42
6,20-03-2019 21:57,4035.46
7,20-03-2019 21:58,4035.43
8,20-03-2019 21:58,4034.76
9,20-03-2019 21:59,4036.15


In [19]:
def count_drop(numbers):
    return len([x for x in numbers if x < 0])

def count_up(numbers):
    return len([x for x in numbers if x > 0])

def actual_calc(row):
    if row['Value'] > row['Open 2']:
        return 0
    return 1

def decider(row):
    if row['Drop 7'] < row['Up 7'] and row['Mean Change 7'] > 0 and row['Change'] >= 0:
        return 1
    elif row['Drop 7'] > row['Up 7'] and row['Mean Change 7'] < 0 and row['Change'] >= 0:
        return 0
    elif row['Drop 7'] < row['Up 7'] and row['Mean Change 7'] > 0 or row['Change'] >= 0:
        return -1
    elif row['Drop 7'] < row['Up 7'] and row['Mean Change 7'] < 0 or row['Change'] < 0:
        return -2
    elif row['Drop 7'] < row['Up 7'] and row['Mean Change 7'] > 0 and row['Change'] < 0:
        return -3
    elif row['Drop 7'] > row['Up 7'] and row['Mean Change 7'] < 0 and row['Change'] < 0:
        return -4
    
def prediction(value):
    if value > 0:
        return 1
    return 0

In [20]:
df['Open 2'] = df['Value'].shift(-1)
df['Max 7'] = df['Value'].rolling(7).max()
df['Min 7'] = df['Value'].rolling(7).min()
df['Change'] = df['Value'] - df['Open 2']
df['Mean Change 7'] = df['Change'].rolling(7).mean()
df['Drop 7'] = df['Change'].rolling(7).apply(count_drop)
df['Up 7'] = df['Change'].rolling(7).apply(count_up)
df['Predict'] = df.apply(decider, axis=1)
df['Prediction'] = df['Predict'].apply(prediction)
df['Actual'] = df.apply(actual_calc, axis=1)

/home/sam/code/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  
/home/sam/code/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


In [22]:
df.head(10)

,Date,Value,Open 2,Max 7,Min 7,Change,Mean Change 7,Drop 7,Up 7,Predict,Prediction,Actual
0,20-03-2019 21:53,4032.37,4036.70,NaN,NaN,-4.33,NaN,NaN,NaN,-2.0,0,1
1,20-03-2019 21:54,4036.70,4036.21,NaN,NaN,0.49,NaN,NaN,NaN,-1.0,0,0
2,20-03-2019 21:54,4036.21,4035.83,NaN,NaN,0.38,NaN,NaN,NaN,-1.0,0,0
3,20-03-2019 21:55,4035.83,4036.39,NaN,NaN,-0.56,NaN,NaN,NaN,-2.0,0,1
4,20-03-2019 21:56,4036.39,4035.42,NaN,NaN,0.97,NaN,NaN,NaN,-1.0,0,0
5,20-03-2019 21:56,4035.42,4035.46,NaN,NaN,-0.04,NaN,NaN,NaN,-2.0,0,1
6,20-03-2019 21:57,4035.46,4035.43,4036.70,4032.37,0.03,-0.437143,3.0,4.0,-1.0,0,0
7,20-03-2019 21:58,4035.43,4034.76,4036.70,4035.42,0.67,0.277143,2.0,5.0,1.0,1,0
8,20-03-2019 21:58,4034.76,4036.15,4036.39,4034.76,-1.39,0.008571,3.0,4.0,-1.0,0,1
9,20-03-2019 21:59,4036.15,NaN,4036.39,4034.76,NaN,NaN,NaN,NaN,NaN,0,1


In [23]:
df[df['Prediction'] != df['Actual']].shape

(6, 12)

In [24]:
df.shape

(10, 12)

In [25]:
df.dropna(inplace=True)

In [26]:
X = df[['Value', 'Max 7', 'Min 7', 'Change', 'Mean Change 7', 'Drop 7', 'Up 7']].values
y = df['Actual'].values

In [27]:
from sklearn.externals import joblib

In [28]:
clf = joblib.load('data/bitcoin-predictor.pkl')

In [29]:
clf.score(X, y)

1.0

In [30]:
df['Actual'].value_counts()

0    2
1    1
Name: Actual, dtype: int64